# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [2]:
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                        FROM orders
                                        WHERE orderDate = '2003-01-31');
                                        """)

jan_31_2004 = pd.DataFrame(cur.fetchall())
jan_31_2004.columns = [x[0] for x in cur.description]
jan_31_2004

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [3]:
cur.execute("""SELECT productName, COUNT(orderNumber), SUM(quantityOrdered)               
               FROM products
               JOIN orderdetails
               USING(productCode)
               GROUP BY productName
               ORDER BY SUM(quantityOrdered) DESC;
               """)

total_orders_by_product = pd.DataFrame(cur.fetchall())
total_orders_by_product.columns = [x[0] for x in cur.description]
total_orders_by_product.head(10)

,productName,COUNT(orderNumber),SUM(quantityOrdered)
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
5,1940s Ford truck,28,1061
6,1969 Harley Davidson Ultimate Chopper,28,1057
7,1957 Chevy Pickup,28,1056
8,1964 Mercedes Tour Bus,28,1053
9,1956 Porsche 356A Coupe,27,1052


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [4]:
cur.execute("""SELECT productName, COUNT(DISTINCT customerNumber) AS unique_customers
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               GROUP BY productName
               ORDER BY unique_customers DESC;
               """)

people_per_product = pd.DataFrame(cur.fetchall())
people_per_product.columns = [x[0] for x in cur.description]
people_per_product.head(10)
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

,productName,unique_customers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
5,HMS Bounty,26
6,Corsair F4U ( Bird Cage),26
7,American Airlines: MD-11S,26
8,1999 Yamaha Speed Boat,26
9,1996 Moto Guzzi 1100i,26


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [28]:
cur.execute("""SELECT DISTINCT employeeNumber, firstName, lastName
               city, officeCode
               FROM employees
               JOIN offices
               USING(officeCode)
               JOIN customers
               ON employeeNumber = salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                     FROM products
                                     JOIN orderdetails
                                     USING(productCode)
                                     JOIN orders
                                     USING(orderNumber)
                                     GROUP BY productCode
                                     HAVING COUNT(DISTINCT customerNumber) < 20
                                     );
                                     """)

emp_sold_slow_sellers = pd.DataFrame(cur.fetchall())
emp_sold_slow_sellers.columns = [x[0] for x in cur.description]
emp_sold_slow_sellers

,employeeNumber,firstName,city,officeCode
0,1370,Gerard,Hernandez,4
1,1501,Larry,Bott,7
2,1337,Loui,Bondur,4
3,1166,Leslie,Thompson,1
4,1286,Foon Yue,Tseng,3
5,1612,Peter,Marsh,6
6,1611,Andy,Fixter,6
7,1401,Pamela,Castillo,4
8,1621,Mami,Nishi,5
9,1323,George,Vanauf,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [32]:
cur.execute("""SELECT employeeNumber, firstName, lastName,
               COUNT(customerNumber) AS num_customers
               FROM employees e          
               JOIN customers c
               ON employeeNumber = salesRepEmployeeNumber
               GROUP BY 1
               HAVING AVG(creditLimit) > 15000;
               """)

emp_with_rich_customers = pd.DataFrame(cur.fetchall())
emp_with_rich_customers.columns = [x[0] for x in cur.description]
emp_with_rich_customers

,employeeNumber,firstName,lastName,num_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!